# Complexity of a Numerical Semigroup functions

## Load libraries

First, we load the package `numericalsgps`.
You can consult the following help pages:
* Listas: https://www.gap-system.org/Manuals/doc/ref/chap21.html#X805CA0B68029B47A
* https://gap-packages.github.io/numericalsgps/doc/chap2.html#X8324E5D97DC2A801

In [ ]:
LoadPackage("numericalsgps");

true

## Function `Ideal2NS`
The function `Ideal2NS` returns the numerical semigroup $\{0\}\cup (sgI + S)$

In [ ]:
Ideal2NS:=function(sgI,S)
    local smallElementsOfI,smallS,NS_I,x,minimumSGI,FS;
    FS:=FrobeniusNumber(S);
    minimumSGI:=Minimum(sgI);
    smallS:=SmallElements(S);
    smallElementsOfI:=[0];
    for x in sgI do
        Append(smallElementsOfI,x+smallS);
    od;
    smallElementsOfI:=Filtered(smallElementsOfI,x->x<=minimumSGI+FS+1);
    smallElementsOfI:=List(Set(smallElementsOfI));
    Sort(smallElementsOfI);
    NS_I:=NumericalSemigroupBySmallElements(smallElementsOfI);
    return NS_I;
end;

function( sgI, S ) ... end

### Example

In [5]:
DeltaNS:=NumericalSemigroup(3,4,5);;
S:=Ideal2NS([5,12],DeltaNS);;
Print(S);

NumericalSemigroup( [ 5, 8, 9, 11, 12 ] )


## Function `isDeltaIdealExtensionOfS`
For $Delta$ and $S$ two numerical semigroups, the function returns true if $Delta$ is an ideal extensions of $S$

In [ ]:
isDeltaIdealExtensionOfS:=function(Delta,S)
    local pfS,msgS,msgDelta,x;
    msgS:=MinimalGeneratingSystemOfNumericalSemigroup(S);
    msgDelta:=MinimalGeneratingSystemOfNumericalSemigroup(Delta);
    for x in msgS do
        if not BelongsToNumericalSemigroup(x,Delta) then
            return false;
        fi;
    od;
    pfS:=PseudoFrobeniusOfNumericalSemigroup(S);
    for x in msgDelta do
        if not BelongsToNumericalSemigroup(x,S) and not x in pfS then
            return false;
        fi;
    od;
    return true;
end;

function( Delta, S ) ... end

### Example

In [ ]:
DeltaNS:=NumericalSemigroup(3,4,5);;
SmallElements(DeltaNS);;
S:=Ideal2NS([5,12],DeltaNS);;
isDeltaIdealExtensionOfS(DeltaNS,S);

true

## Function `idealExtensionsOfNS`
Returns a list all numerical semigroups that are ideal extensions a numerical semigroup $S$.
It uses the auxiliary function:
* nonEmptySubsetsOfA (returns the list of nonempty subsets of a set $A$)
* isPertinenSet (check if a subset of the Frobenius numbers of a numerical semigroup is a pertinent set or not)
* pertinentSets (returns the list of pertinent sets of a numerical semigroup)

In [11]:
nonEmptySubsetsOfA:=function(A)
    local k,lSubsets,lpfs;
    lSubsets:=[];
    for k in [1..Length(A)] do
        lpfs:=Combinations(A,k);
        Append(lSubsets,lpfs);
    od;
    return lSubsets;
end;

function( A ) ... end

In [ ]:
nonEmptySubsetsOfA([11,22,33]);

[ [ 11 ], [ 22 ], [ 33 ], [ 11, 22 ], [ 11, 33 ], [ 22, 33 ], [ 11, 22, 33 ] ]

In [ ]:
isPertinenSet:=function(A,S)
    local pfS,a,b,i,j;
    pfS:=PseudoFrobeniusOfNumericalSemigroup(S);
    if not IsSubsetSet(pfS,A) then 
        return false;
    fi;
    for i in [1..Length(A)] do
        a:=A[i];
        for j in [i..Length(A)] do
            b:=A[j];
            if ((a+b) in pfS) and (not (a+b) in A) then 
                return false;
            fi;
        od;
    od;
    return true;
end;

function( A, S ) ... end

In [16]:
S:=NumericalSemigroup(5,6,7,8,9);
PseudoFrobeniusOfNumericalSemigroup(S);
isPertinenSet([1,4],S);

Numerical semigroup with 5 generators

[ 1, 2, 3, 4 ]

false

In [ ]:
pertinentSets:=function(S)
    local pfS,lSubsetsPFS;
    pfS:=PseudoFrobeniusOfNumericalSemigroup(S);
    lSubsetsPFS:=nonEmptySubsetsOfA(pfS);
    return Filtered(lSubsetsPFS,x->isPertinenSet(x,S));
end;

function( S ) ... end

In [ ]:
pertinentSets(NumericalSemigroup(5,6,7,8,9));

[ [ 3 ], [ 4 ], [ 2, 4 ], [ 3, 4 ], [ 2, 3, 4 ], [ 1, 2, 3, 4 ] ]

In [19]:
idealExtensionsOfNS:=function(S)
    local lpSets,msgS,lIExt,A;
    lIExt:=[];
    lpSets:=pertinentSets(S);
    msgS:=MinimalGeneratingSystemOfNumericalSemigroup(S);
    for A in lpSets do
        Append(lIExt,[NumericalSemigroup(Concatenation(msgS,A))]);
    od;
    return lIExt;
end;

function( S ) ... end

### Examples

In [26]:
S:=NumericalSemigroup(10,17,19);;
Print( idealExtensionsOfNS(S) );

[ NumericalSemigroup( [ 10, 17, 19, 41 ] )
    , NumericalSemigroup( [ 10, 17, 19, 62 ] )
    , NumericalSemigroup( [ 10, 17, 19, 41, 62 ] )
     ]

In [ ]:
S:=NumericalSemigroup(5,6,8,9,10,11,12);;
lDeltas:=idealExtensionsOfNS(S);;
List( lDeltas , x->MinimalGeneratingSystemOfNumericalSemigroup(x) );

[ [ 3, 5 ], [ 4, 5, 6 ], [ 5, 6, 7, 8, 9 ], [ 3, 5, 7 ], [ 4, 5, 6, 7 ], [ 3, 4, 5 ] ]

## Function `chainGamma`
This function returns the list of numerical semigroup that form the chain with the function $\gamma$

In [ ]:
chainGamma:=function(S)
    local lS,a,m,smalls,smalls1,Saux,Nat;
    Nat:=NumericalSemigroup(1);
    lS:=[];
    Saux:=NumericalSemigroup(MinimalGeneratingSystemOfNumericalSemigroup(S));
    while Saux<>NumericalSemigroup(1) do
        a:=Int(Floor( Float(FrobeniusNumber(Saux)/MultiplicityOfNumericalSemigroup(Saux))));
        m:=MultiplicityOfNumericalSemigroup(Saux);
        smalls:=SmallElementsOfNumericalSemigroup(Saux);
        smalls1:=Filtered(smalls,x->x<=a*m);
        Saux:=NumericalSemigroupBySmallElements(smalls1);
        Append(lS,[Saux]);
    od;
    return lS;
end;

function( S ) ... end

### Example

In [32]:
lGamma:=chainGamma(NumericalSemigroup(5,7));;
List(lGamma,x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[ [ 5, 7, 23 ], [ 5, 7, 16, 18 ], [ 5, 7, 11, 13 ], [ 5, 6, 7, 8, 9 ], [ 1 ] ]

## Function `chainWithPFnumbers`

In [143]:
chainWithPFnumbers:=function(S)
    local sg,Nat,Saux,lS,smallsE,a,PFSaux;
    a:=Float(FrobeniusNumber(S)/MultiplicityOfNumericalSemigroup(S));
    sg:=List(MinimalGeneratingSystemOfNumericalSemigroup(S));
    Nat:=NumericalSemigroup(1);
    S:=NumericalSemigroup(sg);
    Saux:=NumericalSemigroup(sg);
    lS:=[S];
    while Saux<>Nat do
        PFSaux:=PseudoFrobeniusOfNumericalSemigroup(Saux);
        smallsE:=List( SmallElementsOfNumericalSemigroup(Saux) );
        Append(smallsE,PFSaux);
        Sort(smallsE);
        Saux:=NumericalSemigroupBySmallElements(smallsE);
        Append(lS,[Saux]);
    od;
    return [lS,Float(Length(lS)-1)=Floor(a)+1];
end;

function( S ) ... end

In [ ]:
S:=NumericalSemigroup(7,8);;
chainpf:=chainWithPFnumbers(S);;
Print( chainpf[1] );
Print( "\nIs the length of this chain equal to the complexity? ", chainpf[2] );

[ ModularNumericalSemigroup( [ 8, 56 ] )
    , NumericalSemigroup( [ 7, 8, 41 ] )
    , NumericalSemigroup( [ 7, 8, 33, 34 ] )
    , NumericalSemigroup( [ 7, 8, 25, 26, 27 ] )
    , NumericalSemigroup( [ 7, 8, 17, 18, 19, 20 ] )
    , NumericalSemigroup( [ 7 .. 13 ] )
    , ModularNumericalSemigroup( [ 1, 2 ] )
     ]
Is the length of this chain equal to the complexity? true

We see how many semigroups with Frobenius number $11$ reach the complexity with kind of chains

In [148]:
List(NumericalSemigroupsWithFrobeniusNumber(11),
    x->chainWithPFnumbers(x)[2]);

[ true, true, true, true, true, false, false, true, true, false, true, true, true, true, true, true, true, true, true, true, true, true, true, false, true, true, true, false, true, false, true, false, false, false, true, true, true, true, true, false, true, false, true, true, true, false, true, true, true, true, true ]

We see that the numerical semigroups of the form $\{ 0, 2k, 3k, 4k,4k+1,\dots\}$ with $k\in\N$ do not reach the complexity

In [ ]:
k:=5;
S:=NumericalSemigroupBySmallElements([0,2*k,3*k,4*k]);
Print( chainWithPFnumbers(S)[2] );

false

5

Numerical semigroup with 10 generators

We show now the numerical with Frobenius number between $2$ and $11$ not verifying this property 

In [ ]:
for n in [2..11] do
    Print("\nn=",n);
    lS:=NumericalSemigroupsWithFrobeniusNumber(n);;
    for S in lS do
        lR:=chainWithPFnumbers(S);
        if lR[2]=false then 
            Print("\n",S);
        fi;
    od;
od;


n=2
n=3
n=4
n=5
n=6
n=7
NumericalSemigroup( [ 4, 6, 9, 11 ] )

n=8
NumericalSemigroup( [ 5, 7, 9, 11, 13 ] )

n=9
NumericalSemigroup( [ 4, 6, 10, 11, 12, 13 ] )

NumericalSemigroup( [ 5, 7, 8, 11 ] )

NumericalSemigroup( [ 5, 8, 11, 12, 14 ] )

NumericalSemigroup( [ 6, 8, 10, 11, 13, 15 ] )

n=10
NumericalSemigroup( [ 6, 8, 9, 11, 13 ] )

NumericalSemigroup( [ 6, 9, 11, 13, 14, 16 ] )

NumericalSemigroup( [ 7, 9, 11, 12, 13, 15, 17 ] )

n=11
NumericalSemigroup( [ 4, 6, 9 ] )

NumericalSemigroup( [ 4, 6, 10, 12, 13, 14, 15 ] )

NumericalSemigroup( [ 4, 10, 12, 13, 14, 15 ] )

NumericalSemigroup( [ 6, 7, 9, 10 ] )

NumericalSemigroup( [ 6, 8, 9, 10, 13 ] )

NumericalSemigroup( [ 6, 8, 10, 13, 15, 17 ] )

NumericalSemigroup( [ 6, 9, 10, 13, 14, 17 ] )

NumericalSemigroup( [ 6, 9, 13, 14, 16, 17 ] )

NumericalSemigroup( [ 6, 10, 13, 14, 15, 17 ] )

NumericalSemigroup( [ 7, 9, 10, 12, 13, 15 ] )

NumericalSemigroup( [ 7, 10, 12, 13, 15, 16, 18 ] )

NumericalSemigroup( [ 8, 10, 12, 13, 14, 

## Function `childrenOfNS`
This function returns the children of a numerical semigroup

In [ ]:
childrenOfNS:=function(S)
    local lChildren,msg,m,fn,a,A,subsets,i,B,M,smalls;
    lChildren:=[];
    msg:=MinimalGeneratingSystemOfNumericalSemigroup(S);
    smalls:=SmallElementsOfNumericalSemigroup(S);
    m:=MultiplicityOfNumericalSemigroup(S);
    fn:=FrobeniusNumber(S);
    a:=Int(Floor(Float(fn)/Float(m))+1)*m;
    A:=Filtered(msg,x->x>a);
    subsets:=nonEmptySubsetsOfA(A);
    #Print(a," A:",A,"\n");
    #Print("subsets:",subsets,"\n");
    for i in [1..Length(subsets)] do
        M:=[(fn+2)..(a+m)];
        SubtractSet(M,subsets[i]);
        B:=List( Union(smalls,M) );
        #Print(smalls," ",M," B:",B,"\n");
        Append(lChildren,[NumericalSemigroupBySmallElements( B )]);
    od;
    return lChildren;
end;

function( S ) ... end

### Examples 

In [ ]:
S:=NumericalSemigroup(3,4,5);;
List(childrenOfNS(S),x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[ [ 3, 5, 7 ], [ 3, 4 ], [ 3, 7, 8 ] ]

In [54]:
S:=NumericalSemigroup(3,5,7);;
List(childrenOfNS(S),x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[ [ 3, 5 ] ]

In [ ]:
S:=NumericalSemigroup(3,4);;
List(childrenOfNS(S),x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[  ]

In [ ]:
S:=NumericalSemigroup(3,7,8);;
List(childrenOfNS(S),x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[ [ 3, 8, 10 ], [ 3, 7, 11 ], [ 3, 10, 11 ] ]

In [60]:
S:=NumericalSemigroup(3,8,10);;
List(childrenOfNS(S),x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[ [ 3, 8, 13 ] ]

In [ ]:
S:=NumericalSemigroup(3,7,11);;
List(childrenOfNS(S),x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[ [ 3, 7 ] ]

In [ ]:
S:=NumericalSemigroup(3,10,11);;
List(childrenOfNS(S),x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[ [ 3, 11, 13 ], [ 3, 10, 14 ], [ 3, 13, 14 ] ]

In [67]:
m:=3;
S:=NumericalSemigroup([m..2*m-1]);
childrenOfNS(S);

3

Numerical semigroup with 3 generators

[ Numerical semigroup, Numerical semigroup, Numerical semigroup ]

## Function `NSWithMultiplicityAndComplexity`
This function returns a list with the numerical semigroups with multiplicity $m$ and complexity $c$. It uses the function `childrenOfNS`

In [ ]:
NSWithMultiplicityAndComplexity:=function(m,c)
    local lS,lChildren,cx,i,lCaux;
    lS:=[NumericalSemigroup([m..2*m-1])];
    #Print(lS,"\n");
    lChildren:=[];
    cx:=1;
    while cx<c do
        for i in [1..Length(lS)] do
            lCaux:=childrenOfNS(lS[i]);
            #Print("lCaux",lCaux,"\n");
            Append(lChildren,lCaux);
        od;
        #Print(lChildren,"\n");
        cx:=cx+1;
        lS:=lChildren;
        lChildren:=[];
    od;
    return lS;
end;

function( m, c ) ... end

### Example

In [ ]:
l34:=NSWithMultiplicityAndComplexity(2,4);;
List(l34,x->MinimalGeneratingSystemOfNumericalSemigroup(x));

[ [ 2, 9 ] ]

In [ ]:
Print( NSWithMultiplicityAndComplexity(3,4) );

[ NumericalSemigroup( [ 3, 8, 13 ] )
    , NumericalSemigroup( [ 3, 7 ] )
    , NumericalSemigroup( [ 3, 11, 13 ] )
    , NumericalSemigroup( [ 3, 10, 14 ] )
    , NumericalSemigroup( [ 3, 13, 14 ] )
     ]